This is the jupyter notebook that is used for the experiments done in the bachelor thesis "Clustering using an Entropy-Modified Gower Similarity Coefficient accounting for Categorical Variability".

Set seed for reproducability and import libraries:

In [ ]:
import numpy as np
from sklearn.metrics import adjusted_rand_score
from sklearn.cluster import SpectralClustering, AgglomerativeClustering
from collections import defaultdict
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.metrics import adjusted_rand_score
from scipy.spatial.distance import squareform
import pandas as pd
from sklearn.datasets import make_blobs
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import time
np.random.seed(42)

#Section 2

Below is a short definition of the Gower similarity and the modification.

First of all we look at the similarity measure proposed by Gower, 1971:      \begin{align}
\mathbf{S}  = \frac{\sum_{k=1}^m s_{ijk}}{\sum_{k=1}^m δ_{ijk}}
\end{align}
where $s_{ijk}$ has three different evaluation possibilities depending on the type of variable we deal with.
If the variable is dichotomous (skip, just means it exists). If it is categorical we use:
\begin{align}
s_{ijk} = \begin{cases}
             1  & \text{if } x_{ik} == x_{jk} \\
             0  & \text{otherwise }
       \end{cases}
\end{align}
If the variable is numerical we use:
\begin{align}
s_{ijk} = 1 - \frac{|x_i - x_j|}{R_k} \quad \text{,where } R_k \text{ is the range of variable k}
\end{align}
 The following change to more accurately weigh categorical variables is proposed. A match in a variable with high variability should account for more similiarity than a match with a low variability one. I try to encorporate entropy to achieve that result. I will transform the $s_{ijk}$ for categorical variables in the following way:
Denote by $U_k=\{1,2,...,P\}$ the amount of unique categories there exist for a given categorical variable $k$ and by $p_i$ the respective relative frequencises, then:
\begin{align}
s_{ijk} = \begin{cases}
    \frac{H(k)}{\log_2{|U_k|}} & \text{if } x_{ik} = x_{jk} \\
    0 & \text{otherwise}.
\end{cases}
\end{align}
For more discussion of this refer to Section 2.2.

After doing a pairwise check and getting a similiarity matrix S of size (n_samples, n_samples), it is normalized, such that the similarity of a given object to itself is 1. Then the similarity matrix is transformed to a distance, using the formula of $D=1-S$.

Implementing it in code:

##Defining Functions

The gower package computes the Gower matrix for a given pandas Dataframe df, allowing for a weight vector w that assigns a weight to each feature. The package also needs a vector indicating which column is a categorical feature.

So in order to compute the modified Gower matrix we have to compute the weights of the categorical features and specify which columns are categorical. The computation of the weights is done in the following code:




In [ ]:
def calculate_entropy(column): # Calculate entropy for a given column
  if len(column)>0:
    x=column.value_counts()
    x=x/len(column)
    ent=0
    for i in range(len(x)):
      ent-=x.iloc[i]*np.log2(x.iloc[i])
    return ent

  else:
    return 0

def calculate_single_weight(column): # Calculate weight for a given column
  return calculate_entropy(column)/np.log2(len(column))

def calculate_weights(array,cat_vec): # Calculate the weights for all features, 1 if it numerical and the definition in Section 2.2 or in the text cell above for the modification
  w=[]
  for i in range(array.shape[1]):
    if cat_vec[i]==True:
      w.append(calculate_single_weight(array[array.columns[i]]))
    else:
      w.append(1)
  return w

In [ ]:
!pip install gower -qU # Make sure the gower package is installed
import gower

#Section 3

## Section 3.1: Creating a Mixed Synthetic Dataset

The following function is used to create a **mixed synthetic dataset** by utilizing the `make_blobs` function from `sklearn`. This function creates data points for fixed hyperparameters.

### Parameters:

- **n_samples**: The number of data points to create.
- **n_features**: The number of initial numerical features.
- **centers**: The number of centers for clustering (similar to the concept of classes).
- **cluster_std**: The amount of variance for a given center. This can be either:
  - A list, where each element represents the variance for each center.
  - A single number, representing the variance for all centers.
- **n_bins**: The number of bins to divide each feature into using the `KBinsDiscretizer` function. This essentially splits the whole range of numbers into categories by assigning a category if a number belongs to a sub-interval.
- **n_discrete**: Specifies the number of initial numerical features to discretize.
- **n_uniform**: The number of features to add with a uniform distribution.
- **n_categories**: The number of categories that the uniform distribution can pick from.

In [ ]:
def create_mixed_synthetic_data(n_samples=5000, n_features=5, centers=4, cluster_std=3.0,
                                n_bins=4, n_discrete=2, n_uniform=2, n_categories=4, random_state=42):
    # Generate the dataset
    data, labels = make_blobs(n_samples=n_samples, n_features=n_features, centers=centers,
                              cluster_std=cluster_std, random_state=random_state)

    # Convert the numpy array to pandas dataframe
    df = pd.DataFrame(data, columns=[f'feature_{i+1}' for i in range(n_features)])

    # Discretize the last n_discrete columns into categorical variables
    if n_discrete > 0:  # Check if n_discrete is greater than 0
        kbd = KBinsDiscretizer(n_bins=n_bins, encode='ordinal', strategy='uniform')
        df_discrete = kbd.fit_transform(df[df.columns[-n_discrete:]])

        df_discrete = pd.DataFrame(df_discrete, columns=[f'feature_{i+1}_disc' for i in range(n_features - n_discrete, n_features)])
        df = pd.concat([df[df.columns[:-n_discrete]], df_discrete], axis=1)

    # For each categorical feature generated from uniform distribution
    for i in range(n_features + 1, n_features + n_uniform + 1):
        df[f'feature_{i}_unif'] = np.random.choice([chr(j) for j in range(65, 65 + n_categories)], size=n_samples)

    # Adding the labels
    df['label'] = labels

    return df

# Example Dataframe
df = create_mixed_synthetic_data()
print(df.head())

   feature_1  feature_2  feature_3  feature_4_disc  feature_5_disc  \
0  -9.989010   6.117292   9.527378             2.0             0.0   
1  -3.076954  10.791599  10.099263             1.0             2.0   
2  -2.503267  13.612906   0.604835             2.0             1.0   
3  -6.208125  -6.628038   9.180104             2.0             3.0   
4  -8.049167  10.140061   4.711128             1.0             1.0   

  feature_6_unif feature_7_unif  label  
0              C              C      0  
1              D              C      2  
2              A              C      0  
3              C              C      1  
4              C              D      2  


The `describe_data` function keeps a record of the range of the numerical features, the distribution of entries for categorical features and the number of bins that were used for discretization.

In [ ]:
def describe_data(df, n_discrete=2, n_uniform=2, n_bins=4):
    # Determine the number of each type of columns
    n_numerical = df.shape[1] - n_discrete - n_uniform - 1  # minus 1 for the label column

    # Define numerical and categorical columns
    num_columns = df.columns[:n_numerical]
    cat_columns = df.columns[n_numerical:-1]  # Exclude the label column

    # Initialize dictionary to store results
    data_description = {}

    # Record range for numerical columns
    for col in num_columns:
        data_description[col] = {
            'min': df[col].min(),
            'max': df[col].max()
        }

    # Record value counts for categorical columns
    for col in cat_columns:
        data_description[col] = {
            'value_counts': df[col].value_counts().to_dict()
        }

    return data_description

#Section 3.5: Finding the best Linkage

 The function for clustering given the computed Gower matrix and linkage parameter is defined.

In [ ]:
def gower_ari(gow, df, centers=4,linkage_method='ward', criterion="maxclust"):
    # Extract the labels
    labels = df['label']

    # Perform hierarchical clustering
    Zd = linkage(squareform(gow), method=linkage_method)

    # Form flat clusters from the hierarchical clustering defined by the linkage matrix
    if criterion=="maxclust":
      cld1 = fcluster(Zd, centers, criterion=criterion)

    # Compute Adjusted Rand index
    ari = adjusted_rand_score(labels, cld1)

    return ari, cld1

Here the clustering result of the baseline using Agglomerative Clustering and the one-hot encoded version of the given dataframe is recorded.

In [ ]:
def cluster_data_agg(df, centers=4, n_features=5, n_discrete=2, n_uniform=2):
    # Extract the features and labels
    X = df.drop(columns=['label'])
    y = df['label']

    # Calculate n_categorical from the sum of n_discrete and n_uniform
    n_categorical = n_discrete + n_uniform

    # Transform the data
    transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(), list(range(n_features-n_discrete, n_features + n_uniform)))],
                                    remainder='passthrough', sparse_threshold=0)
    X_transformed = transformer.fit_transform(X)

    # Initialize and fit AgglomerativeClustering
    agg = AgglomerativeClustering(n_clusters=centers)
    agg.fit(X_transformed)

    # Get cluster labels
    cluster_labels = agg.labels_

    # Compute Adjusted Rand index
    ari = adjusted_rand_score(y, cluster_labels)

    return ari, cluster_labels

## Function: `compare_clustering_methods`

This function evaluates various linkage methods on a given synthetic mixed dataset. It uses the Gower distance matrix and Agglomerative Clustering to compute the Adjusted Rand Index (ARI) for each parameter setting.

### Parameters
- **data**: The input dataset as a pandas DataFrame.
- **n_features, n_discrete, n_uniform, centers, cluster_std, n_bins**: Parameters that may be used when generating synthetic data outside this function.

### Function Process
1. **Linkage Methods and Criteria**: Defines a list of linkage methods ('centroid', 'single', 'complete', 'average', 'ward') for hierarchical clustering.
2. **Configurations**: Specifies configurations for the Gower method, focusing on weighting options (False, True).
3. **Gower Distance Matrix**: Computes the Gower distance matrix once for each dataset and configuration.
4. **ARI Calculation**: For each configuration, the function calculates the ARI for different linkage methods using the Gower distance matrix and stores them.
6. **Result Compilation**: Merges ARI results from the Gower and Agglomerative methods.

### Return
- **results**: A dictionary containing ARI scores for the tested linkages.

In [ ]:
def compare_clustering_methods(data,n_features=5, n_discrete=2, n_uniform=2, centers=4, cluster_std=4,n_bins=4):
    # Create mixed synthetic data
    if isinstance(data, pd.DataFrame):
        df = data

    # Define the linkage methods and criteria to compare
    linkage_methods = ['centroid', 'single', 'complete', 'average', 'ward']

    # Define the weighting and scaling configurations for the Gower method
    configs = [False,True]

    # Extract the features
    X = df.drop(columns=['label']).values

    # Calculate n_categorical from the sum of n_discrete and n_uniform
    n_categorical = n_discrete + n_uniform
    cat_vec = [False] * (n_features - n_discrete) + [True] * n_categorical

    # Calculate ARI for each combination of parameters using the Gower method
    results_gower = {}
    for config in configs:
        weighting = config

        # Compute the Gower distance matrix only once for each dataset
        if weighting:
            w = np.array(calculate_weights(df.drop(columns=['label']), cat_vec))
            gow = gower.gower_matrix(X, cat_features=cat_vec, weight=w)
        else:
            gow = gower.gower_matrix(X, cat_features=cat_vec)

        for linkage_method in linkage_methods:
            ari, _ = gower_ari(gow, df, centers=centers, weighting=weighting, linkage_method=linkage_method)
            results_gower[(linkage_method, weighting)] = ari


    # Merge the results dictionaries and return
    results = {**results_gower}
    return results

## Function: compare_on_random_datasets



This function evaluates the introduced linkage methods on multiple randomly generated datasets. It utilizes the previously described `create_mixed_synthetic_data` and `compare_clustering_methods` functions.

### Parameters

The parameter ranges are fixed according to Section 3.3.
- **n_trials**: Number of random datasets to be generated for the evaluation.
- **min_features, max_features**: Range for the random number of features to be used in the synthetic dataset.
- **min_discrete, max_discrete**: Range for the random number of discrete features in the synthetic dataset.
- **min_uniform, max_uniform**: Range for the random number of features with a uniform distribution in the synthetic dataset.
- **min_centers, max_centers**: Range for the random number of centers for clustering in the synthetic dataset.
- **cluster_std_range**: A tuple indicating the range for cluster standard deviation.

### Internal Variables and Steps

1. **total_ari_scores**: A defaultdict to hold the summed ARIs for each method.

2. **dataset_descriptions**: A list to store the descriptions of each generated dataset.

3. **individual_ari_scores**: A list to store ARI scores for individual datasets.

4. A loop iterates `n_trials` times. Inside the loop:

    a. Random parameters are generated within the specified ranges.
    
    b. A synthetic dataset is generated using the `create_mixed_synthetic_data` function.
    
    c. The description of the generated dataset is recorded.
    
    d. ARI scores are calculated using different clustering methods through the `compare_clustering_methods` function. The scores are updated in the `total_ari_scores` dictionary, and individual scores for this dataset are stored in `individual_dataset_ari_scores`.
    
    e. Individual scores for the dataset are appended to `individual_ari_scores`.

5. Finally, average ARI scores for each method are calculated and stored in `avg_ari_scores`.

### Return

- **avg_ari_scores**: A dictionary containing the average ARI scores for different configurations and clustering methods.
- **dataset_descriptions**: A list containing the descriptions of each generated dataset.
- **individual_ari_scores**: A list containing dictionaries of individual ARI scores for each dataset.

In [ ]:
def compare_on_random_datasets(n_trials=100, min_features=2, max_features=10,
                               min_discrete=0, max_discrete=5,
                               min_uniform=0, max_uniform=5,
                               min_centers=2, max_centers=10,
                               min_bins=2, max_bins=10,
                               cluster_std_range=(1, 5)):

    # Dictionary to hold the summed ARIs for each method and dataset descriptions
    total_ari_scores = defaultdict(float)
    dataset_descriptions = []
    individual_ari_scores = []  # To store ARI scores for individual datasets

    # Loop to generate different standard deviations and perform clustering comparison
    for _ in range(n_trials):
        # Generate random parameters
        n_features = np.random.randint(min_features, max_features + 1)
        n_discrete = np.random.randint(min_discrete, min(n_features, max_discrete + 1))
        n_uniform = np.random.randint(min_uniform, max_uniform + 1)
        centers = np.random.randint(min_centers, max_centers + 1)
        cluster_std = np.random.uniform(cluster_std_range[0], cluster_std_range[1], centers)
        n_bins=int(np.random.uniform(min_bins, max_bins+1))

        # Generate the synthetic dataset
        df = create_mixed_synthetic_data(n_samples=5000, n_features=n_features, centers=centers,
                                         cluster_std=cluster_std, n_bins=n_bins,
                                         n_discrete=n_discrete, n_uniform=n_uniform,
                                         n_categories=4, random_state=42)

        # Record the dataset description
        description = describe_data(df, n_discrete=n_discrete, n_uniform=n_uniform)
        description['std_devs'] = cluster_std.tolist()  # Add standard deviations to the description
        dataset_descriptions.append(description)

        # Initialize a dictionary to store the individual ARI scores for this dataset
        individual_dataset_ari_scores = {}

        # Get the ARI scores for the different clustering methods and update the total scores
        ari_scores = compare_clustering_methods(df,n_features=n_features, n_discrete=n_discrete, n_uniform=n_uniform,
                                                centers=centers, cluster_std=cluster_std, n_bins=4)

        for method, ari in ari_scores.items():
            total_ari_scores[method] += ari
            individual_dataset_ari_scores[method] = ari

        # Append the individual scores for this dataset to the main list
        individual_ari_scores.append(individual_dataset_ari_scores)

    # Divide the total scores by the number of trials to get the average score for each method
    avg_ari_scores = {method: score / n_trials for method, score in total_ari_scores.items()}

    return avg_ari_scores, dataset_descriptions, individual_ari_scores

In this code cell, the modified `compare_on_random_datasets` function is called with 100 trials (`n_trials=100`). This function generates 100 random synthetic datasets and evaluates different configurations of clustering methods on them. The average Adjusted Rand Index (ARI) scores across all the datasets are stored in the variable `average`. The results are reported in Section 3.5.2.

In [ ]:
max_features=15
average, description, ind=compare_on_random_datasets(n_trials=100,max_features=max_features,max_discrete=max_features, max_uniform=8)

In [21]:
average

{('centroid', False): 0.05922121056941135,
 ('single', False): 0.008368815370272658,
 ('complete', False): 0.3210377249774347,
 ('average', False): 0.4258865857256059,
 ('ward', False): 0.509187433502024,
 ('centroid', True): 0.20252709274608247,
 ('single', True): 0.010770037707485709,
 ('complete', True): 0.5937625259463986,
 ('average', True): 0.5536079550074489,
 ('ward', True): 0.7942905005032825}

#Section 3.7

## Modifications to the `compare_on_random_datasets` Function

The code cell contains an adapted version of the `compare_on_random_datasets` function. The main structure of the function remains intact, focused on generating synthetic datasets and evaluating the introduced clustering methods in Section 3. Here are the notable differences from the original version:

### Focused Configuration Testing
The function now specifically targets a subset of configurations for the Gower method. It narrows down the testing scope to configurations using the `ward` linkage method. This is achieved by defining the `gower_configs` list with the configurations of interest.

### Individual Score Recording
The function now captures individual Adjusted Rand Index (ARI) scores for each dataset and configuration. These are stored in a dictionary named `individual_dataset_ari_scores`, which is subsequently appended to a list called `individual_ari_scores` after each trial.

### HDBSCAN Implementation
The function includes the unsupervised HDBSCAN clustering algorithm, applied both directly to the dataset and using a Gower matrix as a distance metric.

### Spectral Clustering
Spectral clustering is also added to the set of methods for evaluation, employing the Gower matrix for calculation.

### Additional Baseline Clustering
The function now includes a baseline using Agglomerative Clustering. This provides another reference point for evaluating clustering methods.

### Enhanced Output
The function continues to return the average ARI scores, dataset descriptions, and individual ARI scores, providing a detailed evaluation of each clustering method's performance on synthetic datasets.


In [ ]:
!pip install hdbscan -qU # Make sure hdbscan library is installed
import hdbscan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 48.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
def spectral_clustering2(gow=None, df=None, centers=4): # Perform Spectral Clustering
  clustering = SpectralClustering(n_clusters=centers, affinity='precomputed').fit(gow)
  ari = adjusted_rand_score(clustering.labels_, df["label"])
  return ari

In [ ]:
def compare_on_random_datasets(n_trials=100, min_features=2, max_features=10,
                               min_discrete=0, max_discrete=5,
                               min_uniform=0, max_uniform=5,
                               min_centers=2, max_centers=10,
                               min_bins=2, max_bins=10,
                               cluster_std_range=(1, 5)):

    total_ari_scores = defaultdict(float)
    dataset_descriptions = []
    individual_ari_scores = []


    for _ in range(n_trials):
        if _ % 10 == 0: # Keep track of how far the program is since it takes a while
            print(_)

        # Existing random dataset generation code
        n_features = np.random.randint(min_features, max_features + 1)
        n_discrete = np.random.randint(min_discrete, min(n_features, max_discrete + 1))
        n_uniform = np.random.randint(min_uniform, max_uniform + 1)
        centers = np.random.randint(min_centers, max_centers + 1)
        cluster_std = np.random.uniform(cluster_std_range[0], cluster_std_range[1], centers)
        n_bins=int(np.random.uniform(min_bins, max_bins+1))

        df = create_mixed_synthetic_data(n_samples=5000, n_features=n_features, centers=centers,
                                         cluster_std=cluster_std, n_bins=n_bins,
                                         n_discrete=n_discrete, n_uniform=n_uniform,
                                         n_categories=4, random_state=42)

        description = describe_data(df, n_discrete=n_discrete, n_uniform=n_uniform, n_bins=n_bins)
        description['std_devs'] = cluster_std.tolist()
        dataset_descriptions.append(description)

        individual_dataset_ari_scores = {}
        X = df.drop(columns=['label']).values

        # Calculating n_categorical and cat_vec as in original code
        n_categorical = n_discrete + n_uniform
        cat_vec = [False] * (n_features - n_discrete) + [True] * n_categorical

        gower_configs = [('ward', 'maxclust', False),
                         ('ward', 'maxclust', True)]


        gower_matrices = {}
        for config in gower_configs:
            if config[2] not in gower_matrices:
                # Compute Gower matrix
                if config[2]:
                    w = np.array(calculate_weights(df.drop(columns=['label']), cat_vec))
                    gower_matrices[(config[2])] = gower.gower_matrix(X, cat_features=cat_vec, weight=w)
                else:
                    gower_matrices[(config[2])] = gower.gower_matrix(X, cat_features=cat_vec)

            gow = gower_matrices[(config[2])]

            # HAC with Ward
            ari, _ = gower_ari(gow, df,centers=centers, linkage_method=config[0])
            method_name = f'gower_{config[0]}_{config[1]}_weighting={config[2]}'
            total_ari_scores[method_name] += ari
            individual_dataset_ari_scores[method_name] = ari

            # Spectral Clustering
            sim=1-gow      #transform back to similarity matrix for spectral clustering
            ari=spectral_clustering2(gow=sim, df=df, centers=centers) #perform spectral clustering
            method_name = f'spectral_weighting={config[2]}'
            total_ari_scores[method_name] += ari

            individual_dataset_ari_scores[method_name] = ari
            # Perform HDBSCAN with Gower matrix
            gow_double = gow.astype(np.float64)
            hdbscan_cluster = hdbscan.HDBSCAN(metric='precomputed')
            labels = hdbscan_cluster.fit_predict(gow_double)
            ari = adjusted_rand_score(df['label'], labels)
            method_name_hdbscan = f'HDBSCAN_weighting={config[2]}'
            total_ari_scores[method_name_hdbscan] += ari
            individual_dataset_ari_scores[method_name_hdbscan] = ari
        # HDBSCAN baseline
        if any(cat_vec):
            encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
            X_categorical = encoder.fit_transform(X[:, np.array(cat_vec)])
            X_numerical = X[:, ~np.array(cat_vec)]
            X_combined = np.hstack([X_numerical, X_categorical])
        else:
            X_combined = X  # Use the original dataset if no categorical variables

        hdbscan_cluster = hdbscan.HDBSCAN()
        labels = hdbscan_cluster.fit_predict(X_combined)
        ari = adjusted_rand_score(df['label'], labels)
        method_name_hdbscan_direct = 'HDBSCAN_baseline'
        total_ari_scores[method_name_hdbscan_direct] += ari
        individual_dataset_ari_scores[method_name_hdbscan_direct] = ari
        # Agglomerative Clustering baseline
        ari, _ = cluster_data_agg(df, centers=centers,
                                  n_features=n_features, n_discrete=n_discrete,
                                  n_uniform=n_uniform)
        total_ari_scores['AgglomerativeClustering_baseline'] += ari
        individual_dataset_ari_scores['AgglomerativeClustering_baseline'] = ari

        individual_ari_scores.append(individual_dataset_ari_scores)

    avg_ari_scores = {method: score / n_trials for method, score in total_ari_scores.items()}

    return avg_ari_scores, dataset_descriptions, individual_ari_scores

The next cell runs the function for 200 trials:

In [ ]:
max_features=15
average,description,individual=compare_on_random_datasets(n_trials=1, max_features=max_features,max_discrete=max_features, max_uniform=8)

In [ ]:
average # Average ARI of each clustering method

{'gower_ward_maxclust_weighting=False': 0.5459322209873017,
 'spectral_weighting=False': 0.5106786723179865,
 'HDBSCAN_weighting=False': 0.1804563465755983,
 'gower_ward_maxclust_weighting=True': 0.7331035637501517,
 'spectral_weighting=True': 0.7121260862011384,
 'HDBSCAN_weighting=True': 0.4190600636963385,
 'AgglomerativeClustering_baseline': 0.7336948915629271,
 'HDBSCAN_baseline': 0.4306307965240029}

In [ ]:
# When is Spectral Clustering better with original Gower similarity matrix than with modification
indices_spectral = [i for i in range(len(individual))
                  if individual[i]['spectral_weighting=False'] >
                  individual[i]['spectral_weighting=True']]
# When is HAC with original Gower distance matrix better than modification
indices_case_gower = [i for i in range(len(individual))
                  if individual[i]['gower_ward_maxclust_weighting=False'] >
                  individual[i]['gower_ward_maxclust_weighting=True']]
# When is HDBSCAN better with original Gower distance matrix than with modification
indices_hdbscan = [i for i in range(len(individual))
                  if individual[i]['HDBSCAN_weighting=False'] >
                  individual[i]['HDBSCAN_weighting=True']]

## Function: `compute_mean_params`

This function computes the mean attributes of dataset descriptions and performance scores for various algorithms. It processes the mean count of different feature types (discrete, uniform, numerical) along with their standard deviations. It also calculates mean performance scores for specific clustering algorithms. It is used to find cases, where clustering using the original Gower matrix outperformed clustering using the modified one.

### Parameters
- **indices**: A list of indices representing specific datasets.
- **description_list**: A list containing dictionaries that describe features of datasets.
- **individual_list**: A list of dictionaries detailing performance scores for different clustering algorithms.

### Function Process
1. Initializes variables for feature counts, standard deviations, and performance scores.
2. Iterates through dataset indices to accumulate feature counts, standard deviations, and algorithm scores.
3. Calculates mean and percentage-based statistics.
4. Stores all results in a Pandas dataframe for easy manipulation.

### Return
A Pandas dataframe containing the following columns:
- **num_indices**: The number of indices provided.
- **mean_num_features**: The mean number of total features.
- **mean_discrete_features**: The mean number of discrete features.
- **mean_uniform_features**: The mean number of uniform features.
- **mean_std_dev**: The mean standard deviation across all features.
- **mean_max_std_dev**: The mean of the maximum standard deviations across all features.
- **mean_min_std_dev**: The mean of the minimum standard deviations across all features.
- **percentage_categorical_features**: The mean percentage of categorical (discrete and uniform) features.
- **percentage_numerical_features**: The mean percentage of numerical features.
- **mean_[algorithm_key]**: The mean performance score for each algorithm, where `algorithm_key` is dynamically identified.

In [ ]:
def compute_mean_params(indices, description_list, individual_list):
    # Variables to store the various counts
    std_dev_means = []
    std_dev_maxs = []
    std_dev_mins = []
    num_discrete_features = 0
    num_uniform_features = 0
    num_num_features = 0

    # Identify all keys dynamically from the first 'individual'
    if individual_list:
        all_keys = list(individual_list[0].keys())
    else:
        all_keys = []

    scores = {key: [] for key in all_keys}

    for i in indices:
        desc = description_list[i]
        individual = individual_list[i]
        std_dev_current = desc['std_devs']

        std_dev_means.append(np.mean(std_dev_current) if std_dev_current else 0)
        std_dev_maxs.append(np.max(std_dev_current) if std_dev_current else 0)
        std_dev_mins.append(np.min(std_dev_current) if std_dev_current else 0)

        num_discrete_features += len([key for key in desc if '_disc' in key])
        num_uniform_features += len([key for key in desc if '_unif' in key])
        num_num_features += len([key for key in desc if '_disc' not in key and '_unif' not in key and key != 'std_devs'])

        for key in all_keys:
            scores[key].append(individual[key])

    mean_std_dev = np.mean(std_dev_means) if std_dev_means else 0
    mean_max_std_dev = np.mean(std_dev_maxs) if std_dev_maxs else 0
    mean_min_std_dev = np.mean(std_dev_mins) if std_dev_mins else 0
    mean_discrete_features = num_discrete_features / len(indices)
    mean_uniform_features = num_uniform_features / len(indices)
    mean_num_features = num_num_features / len(indices)

    total_features = num_discrete_features + num_uniform_features + num_num_features
    percentage_categorical_features = ((num_discrete_features + num_uniform_features) / total_features) * 100
    percentage_numerical_features = (num_num_features / total_features) * 100

    mean_scores = {key: np.mean(score) if score else 0 for key, score in scores.items()}

    data = {
        'num_indices': [len(indices)],
        'mean_num_features': [mean_num_features],
        'mean_discrete_features': [mean_discrete_features],
        'mean_uniform_features': [mean_uniform_features],
        'mean_std_dev': [mean_std_dev],
        'mean_max_std_dev': [mean_max_std_dev],
        'mean_min_std_dev': [mean_min_std_dev],
        'percentage_categorical_features': [percentage_categorical_features],
        'percentage_numerical_features': [percentage_numerical_features],
        **{f"mean_{key}": [mean_score] for key, mean_score in mean_scores.items()}
    }

    df = pd.DataFrame(data)
    return df

In [ ]:
compute_mean_params([x for x in range(len(individual))], description,individual) # Average parameters over all 200 trials

,num_indices,mean_num_features,mean_discrete_features,mean_uniform_features,mean_std_dev,mean_max_std_dev,mean_min_std_dev,percentage_categorical_features,percentage_numerical_features,mean_gower_ward_maxclust_weighting=False_scaling=False,mean_spectral_weighting=False_scaling=False,mean_HDBSCAN_weighting=False_scaling=False,mean_gower_ward_maxclust_weighting=True_scaling=False,mean_spectral_weighting=True_scaling=False,mean_HDBSCAN_weighting=True_scaling=False,mean_gower_ward_maxclust_weighting=True_scaling=True,mean_spectral_weighting=True_scaling=True,mean_HDBSCAN_weighting=True_scaling=True,mean_AgglomerativeClustering
0,200,4.98,3.9,4.155,3.010854,4.333296,1.673985,61.795167,38.204833,0.545932,0.510679,0.180456,0.819667,0.78097,0.389445,0.733104,0.712126,0.41906,0.733695


In [ ]:
compute_mean_params(indices_case_gower, description, individual)

,num_indices,mean_num_features,mean_discrete_features,mean_uniform_features,mean_std_dev,mean_max_std_dev,mean_min_std_dev,percentage_categorical_features,percentage_numerical_features,mean_gower_ward_maxclust_weighting=False_scaling=False,mean_spectral_weighting=False_scaling=False,mean_HDBSCAN_weighting=False_scaling=False,mean_gower_ward_maxclust_weighting=True_scaling=False,mean_spectral_weighting=True_scaling=False,mean_HDBSCAN_weighting=True_scaling=False,mean_gower_ward_maxclust_weighting=True_scaling=True,mean_spectral_weighting=True_scaling=True,mean_HDBSCAN_weighting=True_scaling=True,mean_AgglomerativeClustering
0,4,1.0,11.5,3.25,3.270862,4.395287,1.966623,93.650794,6.349206,0.894633,0.801649,0.27709,0.869301,0.848563,0.329144,0.177913,0.280476,0.226118,0.238455


In [ ]:
compute_mean_params(indices_spectral, description, individual)

,num_indices,mean_num_features,mean_discrete_features,mean_uniform_features,mean_std_dev,mean_max_std_dev,mean_min_std_dev,percentage_categorical_features,percentage_numerical_features,mean_gower_ward_maxclust_weighting=False_scaling=False,mean_spectral_weighting=False_scaling=False,mean_HDBSCAN_weighting=False_scaling=False,mean_gower_ward_maxclust_weighting=True_scaling=False,mean_spectral_weighting=True_scaling=False,mean_HDBSCAN_weighting=True_scaling=False,mean_gower_ward_maxclust_weighting=True_scaling=True,mean_spectral_weighting=True_scaling=True,mean_HDBSCAN_weighting=True_scaling=True,mean_AgglomerativeClustering
0,12,2.5,5.75,4.0,3.075745,4.123308,1.965802,79.591837,20.408163,0.721258,0.795671,0.296839,0.843252,0.763663,0.400606,0.585921,0.620761,0.40599,0.588933


In [ ]:
compute_mean_params(indices_hdbscan, description, individual)

,num_indices,mean_num_features,mean_discrete_features,mean_uniform_features,mean_std_dev,mean_max_std_dev,mean_min_std_dev,percentage_categorical_features,percentage_numerical_features,mean_gower_ward_maxclust_weighting=False_scaling=False,mean_spectral_weighting=False_scaling=False,mean_HDBSCAN_weighting=False_scaling=False,mean_gower_ward_maxclust_weighting=True_scaling=False,mean_spectral_weighting=True_scaling=False,mean_HDBSCAN_weighting=True_scaling=False,mean_gower_ward_maxclust_weighting=True_scaling=True,mean_spectral_weighting=True_scaling=True,mean_HDBSCAN_weighting=True_scaling=True,mean_AgglomerativeClustering
0,25,1.88,5.04,4.8,3.232877,4.253755,2.043039,83.959044,16.040956,0.473591,0.454918,0.197757,0.6756,0.651994,0.175412,0.482945,0.500728,0.230732,0.484181


#Section 4: Compute time

Here the quadratic time complexity for the calculation of the Gower matrix is illustrated using fixed hyperparameters.

In [ ]:
runtime_results = {}

# Generate synthetic data
n_samples = 5000
df = create_mixed_synthetic_data(n_samples=n_samples)

# Extract features and labels
X = df.drop(columns=['label']).values
labels = df['label']

# Calculate n_categorical from the sum of n_discrete and n_uniform
n_features = 5
n_discrete = 2
n_uniform = 2
n_categorical = n_discrete + n_uniform
cat_vec = [False] * (n_features - n_discrete) + [True] * n_categorical

# Measure time for Gower matrix computation with weighting=True
start_time = time()
w =  np.array(calculate_weights(df.drop(columns=['label']), cat_vec)) # Replace with your weight calculation function
gow_weighted = gower.gower_matrix(X, cat_features=cat_vec, weight=w)
runtime_results['With Weighting'] = time() - start_time

# Measure time for Gower matrix computation with weighting=False
start_time = time()
gow_unweighted = gower.gower_matrix(X, cat_features=cat_vec)
runtime_results['Without Weighting'] = time() - start_time

print("Runtimes for different weighting options:", runtime_results)

# Initialize a dictionary to store iterative runtime results for increasing samples
# Initialize dictionaries to store iterative runtime results for increasing samples
iterative_runtime_results_unweighted = {}
iterative_runtime_results_weighted = {}

In [ ]:
# List of sample sizes to iterate over
sample_sizes = [1000, 2000, 4000, 8000, 16000, 32000]

# Iterate through varying sample sizes
for n_samples in sample_sizes:
    df = create_mixed_synthetic_data(n_samples=n_samples)
    X = df.drop(columns=['label']).values
    labels = df['label']

    # Measure time for Gower matrix computation with weighting=False
    start_time = time()
    gow_unweighted = gower.gower_matrix(X, cat_features=cat_vec)
    iterative_runtime_results_unweighted[n_samples] = time() - start_time

    # Measure time for Gower matrix computation with weighting=True
    start_time = time()
    w =  np.array(calculate_weights(df.drop(columns=['label']), cat_vec))
    gow_weighted = gower.gower_matrix(X, cat_features=cat_vec, weight=w)
    iterative_runtime_results_weighted[n_samples] = time() - start_time

print("Runtimes for increasing sample sizes without weighting:", iterative_runtime_results_unweighted)
print("Runtimes for increasing sample sizes with weighting:", iterative_runtime_results_weighted)

In [ ]:
# Convert dictionary keys and values to lists for plotting
unweighted_samples = list(iterative_runtime_results_unweighted.keys())
unweighted_times = list(iterative_runtime_results_unweighted.values())

weighted_samples = list(iterative_runtime_results_weighted.keys())
weighted_times = list(iterative_runtime_results_weighted.values())

# Create the plot
plt.figure(figsize=(10, 6))
plt.plot(unweighted_samples, unweighted_times, marker='o', linestyle='-', label='Without Weighting')
plt.plot(weighted_samples, weighted_times, marker='s', linestyle='-', label='With Weighting')
plt.xlabel('Number of Samples')
plt.ylabel('Runtime (seconds)')
plt.title('Runtime for Computing Gower Distance Matrix')
plt.legend()
plt.grid(True)
plt.show()

#Section 5: Comparison on real life data

Lastly the performance of the used clustering algorithms on the processed Cleveland Heart Disease Dataset is measured. For a more detailed description, look into Section 5.1

In [ ]:
data=pd.read_csv('/content/processed.cleveland.data') # Or whereever it is saved
data.columns=[str(i) for i in data.columns]
labels=data["0"]
data=data.drop(columns="0")

In [ ]:
labels.value_counts() # Distribution of the target classes

0    163
1     55
2     36
3     35
4     13
Name: 0, dtype: int64

In [ ]:
cat_vec=[False, True, True, False, False, True, True, False, True, False, True, False, True] # FTTFFTTFTFTFT corresponding to categorical features
assert len(cat_vec)==len(data.columns)

In [ ]:
for is_cat, col in zip(cat_vec, data.columns): # Transform columns to the right type (numerical, categorical)
    if is_cat:
        data[col] =  data[col].astype('category')
    else:
         data[col] = pd.to_numeric( data[col], errors='coerce')

# Handle missing values
data.replace('?', np.nan, inplace=True)
data.dropna(inplace=True)

In [ ]:
data.shape

(296, 13)

In [ ]:
labels = labels.loc[data.index]

In [ ]:
def gower_ari(gow, labels, linkage_method='ward', criterion='maxclust', t=0.5):
    # Perform hierarchical clustering
    Zd = linkage(squareform(gow), method=linkage_method)

    # Form flat clusters from the hierarchical clustering defined by the linkage matrix
    if criterion == "maxclust":
        cld1 = fcluster(Zd, 5, criterion=criterion)  # Hard-coded centers as 5
    else:
        cld1 = fcluster(Zd, t=t, criterion=criterion)

    # Compute Adjusted Rand index
    ari = adjusted_rand_score(labels, cld1)

    return ari, cld1

In [ ]:
def cluster_data_agg(data, labels, cat_indices):
    # Transform the data
    transformer = ColumnTransformer(transformers=[('cat', OneHotEncoder(), cat_indices)],
                                    remainder='passthrough', sparse_threshold=0)
    X_transformed = transformer.fit_transform(data)

    # Initialize and fit AgglomerativeClustering
    agg = AgglomerativeClustering(n_clusters=5)  # Hard-coded centers as 5
    agg.fit(X_transformed)

    # Get cluster labels
    cluster_labels = agg.labels_

    # Compute Adjusted Rand index
    ari = adjusted_rand_score(labels, cluster_labels)

    return ari, cluster_labels


In [ ]:
def spectral_clustering2(gow=None, df=None, labels=None,centers=5):
  clustering = SpectralClustering(n_clusters=centers, affinity='precomputed').fit(gow)
  ari = adjusted_rand_score(clustering.labels_, labels)
  return ari

In [ ]:
def compare_on_single_dataset(df, labels, cat_vec):
    ari_scores = {}

    X = df.values
    n_features = df.shape[1]

    gower_configs = [('ward', 'maxclust', False),
                     ('ward', 'maxclust', True)]

    for config in gower_configs:
        gow = gower.gower_matrix(X, cat_features=cat_vec) if not config[2] else \
              gower.gower_matrix(X, cat_features=cat_vec, weight=np.array(calculate_weights(df, cat_vec)))
        # HAC with Ward linkage
        ari, _ = gower_ari(gow, labels)
        method_name = f'gower_{config[0]}_{config[1]}_weighting={config[2]}'
        ari_scores[method_name] = ari

        # Spectral Clustering with similarity matrix
        sim = 1 - gow
        ari = spectral_clustering2(gow=sim, labels=labels)
        method_name = f'spectral_weighting={config[2]}'
        ari_scores[method_name] = ari

        # HDBSCAN with Gower matrix
        gow_double = gow.astype(np.float64)
        hdbscan_cluster = hdbscan.HDBSCAN(metric='precomputed')
        labels_pred = hdbscan_cluster.fit_predict(gow_double)#
        print("Amount of data points clustered as noise for weighting=",config[2],": ",np.count_nonzero(labels_pred == -1)) # Print amount of data points HDBSCAN clusters as noise
        ari = adjusted_rand_score(labels, labels_pred)
        method_name_hdbscan = f'HDBSCAN_weighting={config[2]}'
        ari_scores[method_name_hdbscan] = ari

    # HDBSCAN baseline with one-hot encoding
    if any(cat_vec):
            encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
            X_categorical = encoder.fit_transform(X[:, np.array(cat_vec)])
            X_numerical = X[:, ~np.array(cat_vec)]
            X_combined = np.hstack([X_numerical, X_categorical])
    else:
            X_combined = X

    hdbscan_cluster = hdbscan.HDBSCAN()
    cldlabels = hdbscan_cluster.fit_predict(X_combined)
    ari = adjusted_rand_score(labels, cldlabels)

    # Storing the ARI scores for HDBSCAN applied directly to the dataset
    ari_scores['HDBScan_baseline']=ari
    # Agglomerative Clustering baseline with one-hot encoding
    cat_indices = [i for i, is_cat in enumerate(cat_vec) if is_cat]
    ari, _ = cluster_data_agg(df, labels, cat_indices)
    ari_scores['AgglomerativeClustering'] = ari

    return ari_scores

In [ ]:
ari_results = compare_on_single_dataset(data,labels, cat_vec)

Amount of data points clustered as noise for weighting= False :  239
Amount of data points clustered as noise for weighting= True :  216


In [ ]:
ari_results

{'gower_ward_maxclust_weighting=False': 0.14385846540519973,
 'spectral_weighting=False': 0.10372987399375568,
 'HDBSCAN_weighting=False': -0.03607612152284456,
 'gower_ward_maxclust_weighting=True': 0.1949442983641996,
 'spectral_weighting=True': 0.1403257555840935,
 'HDBSCAN_weighting=True': -0.03677007608349537,
 'HDBScan_baseline': 0.08622937741212872,
 'AgglomerativeClustering': 0.0916157114444372}